## Preambule

In [501]:
import numpy as np
import xarray as xr
from importlib import reload
import Plotfunctions
import pandas as pd
import plotly.express as px
from plotly.colors import n_colors
from plotly.subplots import make_subplots
import os
import plotly.figure_factory as ff
import matplotlib as mpl
import plotly.io
from pathlib import Path
import matplotlib.pyplot as plt
from datetime import date
reload(Plotfunctions)
import plotly.graph_objects as go
from Plotfunctions import triangleplot, tableplot, spaghettiplot, draw_line
path = Path.cwd() / "Data"

## With or without C8?

In [502]:
st = ""

## Parameters

In [503]:
timestamp = 2050
srt = 2 # 1=model, 2=ccat

## Prereading stuff

In [504]:
try:
    XRvar.close()
except:
    3
XRvar = xr.open_dataset(path / ("Variances"+st+".nc"))
XRvarsub = XRvar.sel(Time=timestamp)

In [505]:
remove_vars = ["Primary Energy|Other",
               "Primary Energy|Non-Biomass Renewables",
               "Secondary Energy|Electricity|Non-Biomass Renewables",
               "Primary Energy|Ocean",
               "Secondary Energy|Electricity|Ocean",
               "Secondary Energy|Electricity|Other"]
Varlist = pd.read_excel(path / "Varlist.xlsx", sheet_name = "Data")
vars = np.array(Varlist["Variable"])
cats = np.array(Varlist["Category"])
Names = np.unique(cats)
for r in remove_vars:
    wh = np.where((vars != r))[0]
    vars = vars[wh]
    cats = cats[wh]

In [506]:
XRraw = xr.open_dataset(path / ("XRdata"+st+".nc"))
XRmeta = xr.open_dataset(path / ("XRmeta"+st+".nc"))
DF = pd.read_csv(path / ("Models"+st+".csv"))
values = np.array(XRraw.Value)
modscens = np.array(XRraw.ModelScenario)
mods = np.array([i.split('|')[0] for i in modscens])
ccat = np.array(XRmeta.sel(ModelScenario=XRraw.ModelScenario).Category.data)
unimods = np.unique(mods)
uniccat = np.unique(ccat)

In [507]:
XR = XRvar.sel(Variable = vars[cats == "Primary Energy"])
DF_counts = pd.read_csv("X:/user/dekkerm/Projects/AR6_Variance/variancedecomposition/Data/Counts"+st+".csv", index_col=0)
XR = XR.sel(Time=range(2030,2101))
varlist = XR.Variable.data
years = XR.Time.data
varmax = np.max(XR['Var_total']).data
cols = ['forestgreen', 'tomato', 'steelblue', 'goldenrod', 'purple', 'grey', 'brown',
        'magenta', 'red', 'darkgrey', 'blue', 'black', 'darkgreen']
years_str = np.copy(years).astype(str)
years_str[(years_str != '2050') & (years_str != '2100')] = ''

In [508]:
tot = np.array(XR.sel(Time=timestamp).S_c)+np.array(XR.sel(Time=timestamp).S_z)+np.array(XR.sel(Time=timestamp).S_m)
ar = np.array([np.array(XR.Variable),
np.array(XR.sel(Time=timestamp).S_m/tot).round(2),
np.array(XR.sel(Time=timestamp).S_c/tot).round(2),
np.array(XR.sel(Time=timestamp).S_z/tot).round(2)])

colors = n_colors('rgb(255, 200, 200)', 'rgb(200, 0, 0)', 100, colortype='rgb')
bar = px.colors.diverging.RdBu_r
colors = []
for i in range(len(bar)-1):
        colors = colors+n_colors(px.colors.diverging.RdBu_r[i], px.colors.diverging.RdBu_r[i+1], int(100/len(bar)), colortype='rgb')
colors = colors+n_colors(px.colors.diverging.RdBu_r[i+1], px.colors.diverging.RdBu_r[i+1], int(100/len(bar)), colortype='rgb')
ar100 = (ar[1:]*100).astype(int)

In [509]:
def rescale_size(s, max):
    s /= max
    s *= 1e11
    s = s ** 0.1
    return s

## Create traces

In [510]:
scatters = []
tables = []
namecolors = ['violet', 'forestgreen', 'goldenrod', 'peru', 'teal', 'silver']
symbols = [u'\u2601', u"\u2600", u"\U0001F5F2", u"\u26EE", u'\u26DF', u'\u2302']
namesymbols = ['circle', 'diamond-wide', 'star', 'square', 'cross', 'star-triangle-up', 'diamond-tall']
for n_i, name in enumerate(['Emissions', 'Primary Energy', 'Electricity Generation', 'Sector Industry', 'Sector Transportation', 'Sector Residential']):
    # General stuff
    xrsub = XRvarsub.sel(Variable = vars[cats == name])

    # Table
    tot = 1#np.array(xrsub.S_c)+np.array(xrsub.S_z)+np.array(xrsub.S_m)
    var = np.array(xrsub.Variable)
    if name == 'Electricity Generation':
        for v_i in range(len(var)):
            if var[v_i] == 'Secondary Energy|Electricity':
                var[v_i] = 'Total'
            elif var[v_i] == 'Secondary Energy|Electricity|Gas|w/o CCS':
                var[v_i] = 'Gas (without CCS)'
            elif var[v_i] == 'Secondary Energy|Electricity|Gas|w/ CCS':
                var[v_i] = 'Gas (with CCS)'
            elif var[v_i] == 'Secondary Energy|Electricity|Coal|w/o CCS':
                var[v_i] = 'Coal (without CCS)'
            elif var[v_i] == 'Secondary Energy|Electricity|Coal|w/ CCS':
                var[v_i] = 'Coal (with CCS)'
            elif var[v_i] == 'Secondary Energy|Electricity|Biomass|w/ CCS':
                var[v_i] = 'Biomass (with CCS)'
            elif var[v_i] == 'Secondary Energy|Electricity|Biomass|w/o CCS':
                var[v_i] = 'Biomass (without CCS)'
            else:
                var[v_i] = var[v_i][29:]
    if name == 'Primary Energy':
        for v_i in range(len(var)):
            if var[v_i] == 'Primary Energy':
                var[v_i] = 'Total'
            else:
                var[v_i] = var[v_i][15:]
    if name == 'Emissions':
        for v_i in range(len(var)):
            if var[v_i] == 'Emissions|CO2|Energy|Supply|Electricity':
                var[v_i] = 'Electricity (CO<sub>2</sub>)'
            elif var[v_i] == 'Emissions|CO2':
                var[v_i] = 'CO<sub>2</sub>'
            elif var[v_i] == 'Emissions|N2O':
                var[v_i] = 'N<sub>2</sub>O'
            elif var[v_i] == 'Emissions|CH4':
                var[v_i] = 'CH<sub>4</sub>'
            elif var[v_i] == 'Emissions|CO2|Energy|Demand|Industry':
                var[v_i] = 'Industry (CO<sub>2</sub>)'
            elif var[v_i] == 'Emissions|CO2|Energy|Demand|Transportation':
                var[v_i] = 'Transport (CO<sub>2</sub>)'
            elif var[v_i] == 'Emissions|CO2|Energy|Demand|Residential and Commercial':
                var[v_i] = 'Buildings (CO<sub>2</sub>)'
            else:
                var[v_i] = var[v_i][10:]
    if name == 'Sector Industry':
        for v_i in range(len(var)):
            if var[v_i] == 'Final Energy|Industry':
                var[v_i] = 'Total'
            elif var[v_i] == 'Final Energy|Industry|Solids|Coal':
                var[v_i] = 'Coal'
            elif var[v_i] == 'Final Energy|Industry|Solids|Biomass':
                var[v_i] = 'Biomass'
            else:
                var[v_i] = var[v_i][22:]
    if name == 'Sector Residential':
        for v_i in range(len(var)):
            if var[v_i] == 'Final Energy|Residential and Commercial':
                var[v_i] = 'Total'
            elif var[v_i] == 'Final Energy|Residential and Commercial|Solids|Biomass':
                var[v_i] = 'Biomass'
            else:
                var[v_i] = var[v_i][40:]
    if name == 'Sector Transportation':
        for v_i in range(len(var)):
            if var[v_i] == 'Final Energy|Transportation':
                var[v_i] = 'Total'
            elif var[v_i] == 'Final Energy|Transportation|Liquids|Oil':
                var[v_i] = 'Oil'
            elif var[v_i] == 'Final Energy|Transportation|Liquids|Bioenergy':
                var[v_i] = 'Bioenergy'
            else:
                var[v_i] = var[v_i][28:]
    ar = np.array([var,
                   np.array((xrsub.S_m+0.5*xrsub.S_mc)/tot).round(2),
                   np.array((xrsub.S_c+0.5*xrsub.S_mc)/tot).round(2),
                   np.array(xrsub.S_z/tot).round(2)])
    colors = n_colors('rgb(255, 200, 200)', 'rgb(200, 0, 0)', 100, colortype='rgb')
    bar = px.colors.diverging.RdBu_r
    colors = []
    for i in range(len(bar)-1):
        colors = colors+n_colors(px.colors.diverging.RdBu_r[i], px.colors.diverging.RdBu_r[i+1], int(100/len(bar)), colortype='rgb')
    colors = colors+n_colors(px.colors.diverging.RdBu_r[i+1], px.colors.diverging.RdBu_r[i+1], int(100/len(bar)), colortype='rgb')
    ar100 = 10+(ar[1:]*70).astype(int)+10

    head = name
    if name == 'Sector Residential':
        head = "Buildings"
    if name == 'Sector Industry':
        head = "Industry"
    if name == 'Sector Transportation':
        head = "Transport"
    if name == 'Electricity Generation':
        head = "Electricity"
    tables.append(go.Table(columnwidth = [4, 1, 1, 1],
                        header = dict(values = [['<b>'+head+'</b>'],
                                      ["<b>Model</b>"],
                                      ['<b>Climate</b>'],
                                      ['<b>Other</b>']],
                                      line_color='white',
                                      fill_color=[namecolors[n_i], 'white', 'white', 'white'],
                                      align=['center','center','center','center'],
                                      font=dict(color='black', size=12),
                                      height=22,
                                     ),
                        cells=dict(values=ar[:, ar[srt].argsort()[::-1]],
                                   line={'color': 'black', 'width': 0},
                                   align=['center','center','center','center'],
                                   fill_color=['white', 
                                               np.array(colors)[ar100[:, ar[srt].argsort()[::-1]][0]],
                                               np.array(colors)[ar100[:, ar[srt].argsort()[::-1]][1]],
                                               np.array(colors)[ar100[:, ar[srt].argsort()[::-1]][2]]],
                                   font={#'family' : ["Arial", "Arial Black", "Arial Black", "Arial Black"],
                                         'color': 'black',
                                         'size': 11},
                                   height=22,
                                  )
                       ))

    # Scatter
    varlist = np.array(xrsub.Variable)
    for v in range(len(varlist)):
            ds = xrsub.sel(Variable=varlist[v])
            a, b, c, d, s = np.array(ds[['S_c', 'S_m', 'S_z', 'S_mc', 'Var_total']].to_array())
            s = 20# 2*rescale_size(float(s/XRraw.sel(Time=timestamp, Variable=varlist[v]).Value.mean(axis=0)), varmax)
            lw = 1 # s/10
            if varlist[v] == 'Secondary Energy|Electricity|Wind+Solar':
                    count = np.min([list(DF_counts[DF_counts.Variable=="Secondary Energy|Electricity|Wind"].Count)[0],
                                    list(DF_counts[DF_counts.Variable=="Secondary Energy|Electricity|Solar"].Count)[0]])
            elif varlist[v] == 'Primary Energy|Wind+Solar':
                    count = np.min([list(DF_counts[DF_counts.Variable=="Primary Energy|Wind"].Count)[0],
                                    list(DF_counts[DF_counts.Variable=="Primary Energy|Solar"].Count)[0]])
            else:
                    count = list(DF_counts[DF_counts.Variable==varlist[v]].Count)[0]

            maxsize2 = 32
            if varlist[v] in ['Primary Energy|Coal',
                              'Primary Energy|Oil',
                              'Emissions|Kyoto Gases',
                              'Secondary Energy|Electricity|Coal|w/o CCS',
                              'Final Energy|Industry|Electricity',
                              'Final Energy|Transportation|Electricity',
                              'Final Energy|Residential and Commercial|Electricity',
                              'Final Energy|Transportation|Hydrogen',
                              'Primary Energy|Renewables (incl. Biomass)']:
                da = 0
                if varlist[v] == "Emissions|Kyoto Gases": da = 0.06
                lw = 0
                scatters.append(go.Scatterternary(a=[a+0.5*d+da], b=[b+0.5*d], c=[c],
                                                hoverinfo='skip',
                                                mode='markers', marker={'size': maxsize2-4, 'symbol': 'circle-open', 'color': namecolors[n_i]},
                                                marker_line_width = 2,
                                                showlegend=False))
                scatters.append(go.Scatterternary(a=[a+0.5*d+da], b=[b+0.5*d], c=[c],
                                                hoverinfo='skip',
                                                mode='markers', marker={'size': maxsize2, 'symbol': 'circle-open', 'color': 'black'},
                                                marker_line_width = 3,
                                                showlegend=False))
            scatters.append(go.Scatterternary(a=[a+0.5*d], b=[b+0.5*d], c=[c],
                                            mode='markers', name=varlist[v]+' ('+str(count)+')', showlegend=False, text=[timestamp],
                                            hovertemplate='%{text}<br>Var (climate): %{a}<br>Var (model): %{b} <br>Var (other): %{c}',
                                            marker={'size': 0.1, 'symbol': namesymbols[n_i], 'color': namecolors[n_i], 'opacity': 1, 'line' :dict(width=lw, color='black')},
                                            line={'width': 1.5},
                                            textfont=dict(size=1, color=namecolors[n_i])))
            maxsize = 23
            if name == 'Sector Transportation':
                scatters.append(go.Scatterternary(a=[a+0.5*d], b=[b+0.5*d], c=[c],
                                mode='text', textfont={'size': maxsize-2, 'color': 'teal'},
                                text=[u'\u26DF'], showlegend=False))
            if name == 'Sector Residential':
                for k in np.arange(1, maxsize, 1):
                    scatters.append(go.Scatterternary(a=[a+0.5*d], b=[b+0.5*d], c=[c],
                                    mode='text', textfont={'size': k, 'color': 'darkgray'},
                                    text=[u'\u2302'], showlegend=False))
            if name == 'Electricity Generation':
                scatters.append(go.Scatterternary(a=[a+0.5*d], b=[b+0.5*d], c=[c],
                                mode='text', textfont={'size': maxsize, 'color': 'goldenrod'},
                                text=[u"\U0001F5F2"], showlegend=False))
            if name == 'Sector Industry':
                # for k in np.arange(20, 35, 1):
                #     scatters.append(go.Scatterternary(a=[a], b=[b], c=[c],
                #                     mode='text', textfont={'size': k, 'color': 'peru'},
                #                     text=[u"\u26EE"], showlegend=False))
                scatters.append(go.Scatterternary(a=[a+0.5*d], b=[b+0.5*d], c=[c],
                                mode='text', textfont={'size': maxsize-2, 'color': 'peru'},
                                text=[u"\u26EE"], showlegend=False))
            if name == 'Emissions':
                scatters.append(go.Scatterternary(a=[a+0.5*d], b=[b+0.5*d], c=[c],
                                mode='text', textfont={'size': maxsize, 'color': 'violet'},
                                text=[u'\u2601'], showlegend=False))
            if name == 'Primary Energy':
                scatters.append(go.Scatterternary(a=[a+0.5*d], b=[b+0.5*d], c=[c],
                                mode='text', textfont={'size': maxsize, 'color': 'forestgreen'},
                                text=[u"\u2600"], showlegend=False))
#['Emissions', 'Primary Energy', 'Electricity Generation', 'Sector Industry', 'Sector Transportation', 'Sector Residential']


Ternary plot

In [511]:
trace_ternary_lines = []
for i in range(10):
        a = [0+i*0.1, 0+i*0.1]
        b = [0, 1-a[0]]
        c = [1-a[0], 0]
        rgb = mpl.colors.colorConverter.to_rgb(plt.cm.get_cmap('Greys')(0.+i/20))
        col = 'rgb('+str(rgb[0])+','+str(rgb[1])+','+str(rgb[2])+')'
        trace_ternary_lines.append(go.Scatterternary(a=a, b=b, c=c, showlegend=False, mode='lines', hoverinfo='skip', line={'width': 0.25+i*0.1, 'color': col}))
        trace_ternary_lines.append(go.Scatterternary(a=b, b=a, c=c, showlegend=False, mode='lines', hoverinfo='skip', line={'width': 0.25+i*0.1, 'color': col}))
        trace_ternary_lines.append(go.Scatterternary(a=b, b=c, c=a, showlegend=False, mode='lines', hoverinfo='skip', line={'width': 0.25+i*0.1, 'color': col}))

trace_ternary_border = []
#trace_ternary_border.append(go.Scatterternary(a=[0.005, 0.005], b=[0, 0.25], c=[0.25, 0], showlegend=False, mode='lines', hoverinfo='skip', line={'width': 2, 'color': 'black'}))
#trace_ternary_border.append(go.Scatterternary(a=[0, 0.5], b=[0.005, 0.005], c=[0.5, 0], showlegend=False, mode='lines', hoverinfo='skip', line={'width': 2, 'color': 'black'}))
#trace_ternary_border.append(go.Scatterternary(a=[0, 1], b=[1, 0], c=[0.005, 0.005], showlegend=False, mode='lines', hoverinfo='skip', line={'width': 2, 'color': 'black'}))

C:\Users\dekkerm\AppData\Local\Temp\2\ipykernel_8580\4024270652.py:6: MatplotlibDeprecationWarning:

The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.



## Contours

In [512]:
def clean_data(data_in):
    """
    Cleans data in a format which can be conveniently
    used for drawing traces. Takes a dictionary as the
    input, and returns a list in the following format:

    input = {'key': ['a b c']}
    output = [key, [a, b, c]]
    """
    key = list(data_in.keys())[0]
    data_out = [key]
    for i in data_in[key]:
        data_out.append(list(map(float, i.split(' '))))

    return data_out

In [513]:
conts = []
for i in np.arange(35, 51, 5):
    conts.append({str(i): [str(i) +' '+str(i)+' '+str(100-2*i),
                  str(i) +' '+str(100-2*i)+' '+str(i),
                  str(100-2*i) +' '+str(i)+' '+str(i)]})
for i in np.arange(55, 101, 5):
    i0 = str(i)
    i1 = str(100-i)
    conts.append({str(i): [i0+' '+i1+' 0',
                            i0+' 0 '+i1,
                            i1+' 0 '+i0,
                            '0 '+i1+' '+i0,
                            '0 '+i0+' '+i1,
                            i1+' '+i0+' 0',
                            i0+' '+i1+' 0']})

In [514]:
conts2 = []
for i in np.arange(35, 51, 5):
    conts2.append({str(i): [str(i) +' '+str(i)+' '+str(100-2*i),
                  str(i) +' '+str(100-2*i)+' '+str(i),
                  str(100-2*i) +' '+str(i)+' '+str(i)]})
for x in np.arange(55, 101, 5):
    conts2.append({str(i): [str(x)+' '+'0'+' '+str(100-x),
                               str(x-5)+' '+'0'+' '+str(100-(x-5)),
                               str(x-5)+' '+str(100-(x-5))+' '+'0',
                               str(x)+' '+str(100-x)+' '+'0',
                               str(x)+' '+'0'+' '+str(100-x)
                                ]})
    conts2.append({str(i): ['0'+' '+str(100-x)+' '+str(x),
                               '0'+' '+str(100-(x-5))+' '+str(x-5),
                               str(100-(x-5))+' '+'0'+' '+str(x-5),
                               str(100-x)+' '+'0'+' '+str(x),
                               '0'+' '+str(100-x)+' '+str(x)
                                ]})
    conts2.append({str(i): [str(100-x)+' '+str(x)+' '+'0',
                               str(100-(x-5))+' '+str(x-5)+' '+'0',
                               '0'+' '+str(x-5)+' '+str(100-(x-5)),
                               '0'+' '+str(x)+' '+str(100-x),
                               str(100-x)+' '+str(x)+' '+'0'
                                ]})

In [515]:
bar = px.colors.sequential.Greys # px.colors.diverging.RdBu_r
#bar = bar[2:-2]
#bar = bar[int(len(bar)/2)-2:]
colors = []
for i in range(len(bar)-1):
    colors = colors+n_colors(bar[i], bar[i+1], int(200/len(bar)), colortype='rgb')
colors = colors+n_colors(bar[i+1], bar[i+1], int(200/len(bar)), colortype='rgb')

In [516]:
conttraces = []
n = 0
for raw_data in conts2:
    data = clean_data(raw_data)

    a = [inner_data[0] for inner_data in data[1:]]
    a.append(data[1][0]) # Closing the loop

    b = [inner_data[1] for inner_data in data[1:]]
    b.append(data[1][1]) # Closing the loop

    c = [inner_data[2] for inner_data in data[1:]]
    c.append(data[1][2]) # Closing the loop

    conttraces.append(go.Scatterternary(
        text = data[0],
        a=a, b=b, c=c, mode='lines',
        line=dict(color='black', width=0),
        fill='toself',showlegend=False, hoverinfo='skip', opacity=0.2, 
        fillcolor = colors[int(200/len(conts2)*n)]#colors_iterator.__next__()
    ))
    n+=1
conttraces = conttraces[::-1]

## Add together

In [517]:
def draw_line(fig, name, x, y, xt, yt, color):
    fig.add_shape(type="line", xref='paper', yref='paper',
                  x0=x,
                  y0=x+xt,
                  x1=x,
                  y1=y+yt,
                  line=dict(color=color, width=3))
    fig.add_annotation(xref='paper',
                       yref='paper',
                       #xref = 'x4',
                       #yref = 'y4',
                       x=x,
                       ax=xt,
                       align="center",
                       y=y,
                       ay=yt,
                       text=name,
                       #textangle=90,
                       font=dict(color=color,
                                 family='Arial Black',
                                 size=12),
                       showarrow=True)

In [518]:
def draw_ann(fig, name, x, y, color, vert, hor):
    align = 'left'
    h = 1
    if hor == 'left':
        align = 'right'
        h = -1
    v = 1
    if vert == 'down':
        v = -1
    dy = 0
    if y>0.5:
        dy = 0.02
    dx = 0
    if y> 0.9:
        dx = -0.02
    fig.add_shape(type="line", xref='paper', yref='paper',
                  x0=x,
                  y0=y,
                  x1=x+0.02*h,
                  y1=y+0.02*v,
                  line=dict(color='black', width=3))
    fig.add_shape(type="line", xref='paper', yref='paper',
                  x0=x+[0, 0.15, -0.11][h],
                  y0=y+0.02*v,
                  x1=x+0.02*h,
                  y1=y+0.02*v,
                  line=dict(color='black', width=3))
    fig.add_annotation(xref='paper',
                       yref='paper',
                       #xref = 'x4',
                       #yref = 'y4',
                       x=x+0.2*h+dx,
                       ax=0,
                       align=align,
                       y=y+0.02*v+dy,
                       ay=-0,
                       text=name,
                       #textangle=90,
                       font=dict(color=color,
                                 family='Arial Black',
                                 size=12),
                       showarrow=False)

In [519]:
# -*- coding: utf-8 -*-
fig = make_subplots(
    rows=10, cols=3,
    horizontal_spacing = 0.04,
    vertical_spacing=0,
    # column_widths = [0.15, 0.7, 0.15],
    # row_heights = [0.15, 0.7, 0.15],
    # specs = [[{"type": "table", "rowspan": 2}, {"type": "scatterternary", "colspan": 2, "rowspan": 6}, {}, {"type": "table", "rowspan": 2}],
    #          [{}, {}, {}, {}],
    #          [{"type": "table", "rowspan": 2}, {}, {}, {"type": "table", "rowspan": 2}],
    #          [{}, {}, {}, {}],
    #          [{"type": "table", "rowspan": 2}, {}, {}, {"type": "table", "rowspan": 2}],
    #          [{}, {}, {}, {}]
    #         ]
    specs = [[{"type": "scatterternary", "colspan": 2, "rowspan": 6}, {}, {"type": "table", "rowspan": 2}],
             [{}, {}, {}],
             [{}, {}, {"type": "table", "rowspan": 2}],
             [{}, {}, {}],
             [{}, {}, {"type": "table", "rowspan": 2}],
             [{}, {}, {}],
             [{}, {},  {"type": "table", "rowspan": 4}],
             [{"type": "table", "rowspan": 3}, {"type": "table", "rowspan": 3}, {}],
             [{}, {}, {}],
             [{}, {}, {}]
            ]
)

# Lines 50%
fig.add_trace(go.Scatterternary(a=[0.5, 0.5, 0, 0.5], b=[0, 0.5, 0.5, 0], c=[0.5, 0, 0.5, 0.5],
                                                mode='lines',
                                                showlegend=False,
                                                hoverinfo='skip',
                                                marker={'size': 15,
                                                        'color': 'black',
                                                        'opacity': 1,#0.9-i*0.03,
                                                        'line' :dict(width=0., color='black')},
                                                line={'width': 1, #np.mean(s[i:i+2])*0.5,
                                                        'color': 'black',
                                                        'dash': 'dot'},
                                                textfont=dict(size=5,
                                                        color='black')), 1, 1)

fig.add_trace(tables[0], 8, 1)
fig.add_trace(tables[1], 8, 2)
fig.add_trace(tables[5], 5, 3)

fig.add_trace(tables[3], 1, 3)
fig.add_trace(tables[4], 3, 3)
fig.add_trace(tables[2], 7, 3)


#fig.add_traces(fig1.data, 1, 1)
for n in range(len(conttraces)):
        fig.add_trace(conttraces[n], 1, 1)
# for n in range(len(conttracesb)):
#         fig.add_trace(conttracesb[n], 1, 1)
# for n in range(len(trace_ternary_lines)):
#     fig.add_trace(trace_ternary_lines[n], 1, 1)
# for n in range(len(trace_ternary_border)):
#     fig.add_trace(trace_ternary_border[n], 1, 1)
for n in range(len(scatters)):
    fig.add_trace(scatters[n], 1, 1)
fig.update_layout(height=1000, width=1190)
fig.update_layout({'margin':dict(l=0,r=0,b=0,t=35),
'ternary':
{'sum':1,
'bgcolor':'rgba(0,0,0,0)',#'whitesmoke',
    # "paper_bgcolor":'rgba(0,0,0,0)',
    # "plot_bgcolor":'rgba(0,0,0,0)',
'aaxis':{'title': '', 'min': 0, 
        'linewidth':0, 'ticks':'outside',
        'tickmode':'array','tickvals':[0.5, 0.6, 0.7, 0.8, 0.9, 1.0], 'ticktext':['50%', '60%', '70%', '80%', '90%', '100%'], 'tickfont':{'size':12}},
'baxis':{'title': '', 'min': 0, 
        'linewidth':2, 'ticks':'outside',
        'tickmode':'array','tickvals':[0.5, 0.6, 0.7, 0.8, 0.9, 1.0], 'ticktext':['50%', '60%', '70%', '80%', '90%', '100%'],'tickangle':60, 'tickfont':{'size':12}},
'caxis':{'title': '', 'min': 0, 
        'linewidth':2, 'ticks':'outside',
        'tickmode':'array','tickvals':[0.5, 0.6, 0.7, 0.8, 0.9, 1.0], 'ticktext':['50%', '60%', '70%', '80%', '90%', '100%'],'tickangle':-60, 'tickfont':{'size':12}}}})

# fig.add_trace(go.Scatterternary(a=[a], b=[b], c=[c],
# mode='text', textfont={'size': 35},
# #text=[u'&#128663;'.encode("utf-8")]))
# text=[u'\u9935'.encode("utf-8")]))
#draw_line(fig, "Primary energy coal", 0.5, 0.725, 150, -40, "forestgreen")
# draw_line(fig, "Primary energy oil", 0.49, 0.62, 210, -40, "violet")
# draw_line(fig, "Kyoto gas emissions", 0.51, 0.905, 120, -30, "goldenrod")
# draw_line(fig, "Coal (w/o CCS) in<br>electricity production", 0.52, 0.765, 150, -60, "forestgreen")
# draw_line(fig, "Electricity usage<br>in industry", 0.385, 0.235, -120, -120, "silver")
# draw_line(fig, "Electricity usage<br>in transport", 0.395, 0.205, 200, -40, "teal")
#draw_line(fig, "Electricity usage<br>in residential", 0.38, 0.145, 180, 40, "silver")
if st != '_c78':
        draw_ann(fig, "Electricity in<br>residential sector", 0.19, 0.486, "dimgray", "down", "right")
        draw_ann(fig, "Electricity in<br>transport sector", 0.215, 0.554, "teal", "up", "right")
        draw_ann(fig, "Electricity in<br>industry sector", 0.196, 0.588, "peru", "up", "left")
        draw_ann(fig, "Primary Energy<br>from coal", 0.30, 0.845, "forestgreen", "up", "left")
        draw_ann(fig, "Primary Energy<br>from oil", 0.29, 0.748, "forestgreen", "down", "left")
        draw_ann(fig, "Kyoto gas<br>emissions", 0.343, 0.950, "violet", "down", "right")
        draw_ann(fig, "Electricity from<br>coal (w/o CCS)", 0.335, 0.837, "goldenrod", "down", "right")
        draw_ann(fig, "Primary Energy<br>from renewables", 0.365, 0.7, "forestgreen", "up", "right")
        draw_ann(fig, "Hydrogen use in transport", 0.335, 0.52, "teal", "up", "right")

# TRIANGLE CORNER LABELS
args = dict(xref="paper", yref="paper", align='center', showarrow=False)
fig.add_annotation(text="Climate target", x=0.242, y=1.06, **args, font=dict(color='darkgray', size=25))
fig.add_annotation(text="main driver", x=0.284, y=1.035, **args, font=dict(color='darkgray', size=16))
fig.add_annotation(text="Model differences", x=-0.015, y=0.32, **args, font=dict(color='darkgray', size=25))
fig.add_annotation(text="main driver", x=0.04, y=0.305, **args, font=dict(color='darkgray', size=16))
fig.add_annotation(text="Other scenario", x=0.68, y=0.38, **args, font=dict(color='darkgray', size=25))
fig.add_annotation(text="assumptions", x=0.675, y=0.36, **args, font=dict(color='darkgray', size=25))
fig.add_annotation(text="main driver", x=0.60, y=0.34, **args, font=dict(color='darkgray', size=16))

# ADD SYMBOLS
args = dict(xref="paper", yref="paper", showarrow=False)
for i in range(6):
        x = [0.005, 0.37, 0.735, 0.735, 0.735, 0.735][i]
        y = [0.25, 0.25, 1.03, 0.831, 0.59, 0.39][i]
        #fig.add_annotation(text=u'\u25CB', x=x, y=y, **args, font=dict(color='black', size=100))
        fig.add_annotation(text=u'\u25CF', x=x, y=y, **args, font=dict(color='white', size=60))
        fig.add_annotation(text=symbols[[0, 1, 3, 4, 5, 2][i]],
                           x=x+[0.005, 0, -0.0049, -0.005, -0.01, -0.006][i],
                           y=y+[0.015, 0.018, -0.031, -0.029, -0.002, -0.005][i],
                           **args,
                           font=dict(color=['violet', 'forestgreen', 'peru', 'teal', 'dimgray', 'goldenrod'][i], size=[25, 25, 22, 22, 25, 22][i]))

# AXIS ANNOTATIONS
args = dict(xref="paper", yref="paper", showarrow=False)
fig.add_annotation(text="Fraction of variance<br>determined by driver", x=0.145, y=0.875, textangle=-60, **args, font=dict(color='darkgray', size=12))
fig.update_layout({'margin':dict(l=25,r=7,b=0,t=55)})
fig.show()

## Savings

In [520]:
#st = "_2100"

try:
    os.remove('Figures/Paperfigures/Figure_3'+st+'.html')
except:
    3
def html_w(typ):
    return '<html> '+typ+' <p style="font-family: Arial">'

fig.write_image("Figures/Paperfigures/Figure_3"+st+".pdf")
fig.write_image("Figures/Paperfigures/Figure_3"+st+".png", scale=3)
fig.write_image("Figures/Paperfigures/Figure_3"+st+".svg")
today = date.today()
td = today.strftime("%B %d, %Y")

with open('Figures/Paperfigures/Figure_3'+st+'.html', 'a') as f:
    f.write(html_w('<h1>')+'Figure 3 - Overview</p></h1>')
    f.write(html_w('<body>')+'Data is for year '+str(timestamp)+'. Version: '+td+'</p></body>')
    f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))